In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
# import nbformat
import tensorflow as tf

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic





ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(data):
    data_columns = ['x', 'y', 'z']
    # data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [24]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
# import nbformat
import tensorflow as tf

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic


def create_frame_landmark_df(results, frame, xyz):
    xyz_skel = xyz[['type','landmark_index']].drop_duplicates().reset_index(drop=True).copy()

    face = pd.DataFrame()
    pose = pd.DataFrame()
    left_hand = pd.DataFrame()
    right_hand = pd.DataFrame()
    if results.face_landmarks:
        for i, point in enumerate(results.face_landmarks.landmark):
            face.loc[i, ['x','y','z']] = [point.x, point.y, point.z]
    if results.pose_landmarks:
        for i , point in enumerate(results.pose_landmarks.landmark):
            pose.loc[i, ['x', 'y', 'z']] = [point.x, point.y, point.z]
    if results.left_hand_landmarks:
        for i, point in enumerate(results.left_hand_landmarks.landmark):
            left_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]
    if results.right_hand_landmarks:
        for i, point in enumerate(results.right_hand_landmarks.landmark):
            right_hand.loc[i, ['x', 'y', 'z']] = [point.x, point.y, point.z]   
    face = face.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='face')
    pose = pose.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='pose')
    left_hand = left_hand.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='left_hand')
    right_hand = right_hand.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='right_hand')


    landmarks = pd.concat([face,pose,right_hand,left_hand]).reset_index(drop=True)
    landmarks = xyz_skel.merge(landmarks, on=['type','landmark_index'], how='left')
    landmarks = landmarks.assign(frame=frame)
    return landmarks


# For webcam input:
def do_capture_loop(xyz):
    all_landmarks = []
    count = 0
    cap = cv2.VideoCapture(1)
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
        frame = 0
        while cap.isOpened():
            frame+=1
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)

        ## create landmarks dataframe from results
            landmarks = create_frame_landmark_df(results,frame,xyz)
            all_landmarks.append(landmarks)
        
        # Draw landmark annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                results.face_landmarks,
                mp_holistic.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_pose_landmarks_style())
            #Flip the image horizontally for a selfie-view display.
            
            # text = prediction_func(pq_file)
            # print(text)
            # draw_predictions(image,text)
            # cv2.rectangle(frame, (x, y - text_height - 5), (x + text_width, y), (0, 0, 0), -1)
            # cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))

            if cv2.waitKey(5) & 0xFF == 27:
                break
    # cap.release()
    return pd.concat(all_landmarks).reset_index(drop=True)
pq_file_sample = "train_landmark_files/16069/100015657.parquet"
xyz = pd.read_parquet(pq_file_sample)
data = do_capture_loop(xyz)
# pd.concat(landmarks).reset_index(drop=True).to_parquet('output.parquet')

In [49]:
data = data[['x','y','z']]

In [52]:
# pq_file = "output.parquet"
def prediction_func(data):

    ## defining the model 
    interpreter = tf.lite.Interpreter("model.tflite")
    found_signatures = list(interpreter.get_signature_list().keys())
    prediction_fn = interpreter.get_signature_runner("serving_default")

    
    # Add ordinally Encoded Sign (assign number to each sign name)\
    train = pd.read_csv('train.csv.zip')
    train['sign_ord'] = train['sign'].astype('category').cat.codes

    # Dictionaries to translate sign <-> ordinal encoded sign
    SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
    ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

    ## load data from output parquet
    xyz_np = load_relevant_data_subset(data)
    prediction = prediction_fn(inputs=xyz_np)
    sign = prediction['outputs'].argmax()
    return ORD2SIGN[sign]






In [53]:
prediction_func(data)
# load_relevant_data_subset(data)


'police'

In [39]:
int(len(data)/ROWS_PER_FRAME),int(len(parquet_data)/ROWS_PER_FRAME)

(61, 61)

In [46]:
parquet_data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))


array([[[        nan,         nan,         nan],
        [        nan,         nan,         nan],
        [        nan,         nan,         nan],
        ...,
        [        nan,         nan,         nan],
        [        nan,         nan,         nan],
        [        nan,         nan,         nan]],

       [[ 0.57347399,  0.55271643, -0.04534879],
        [ 0.57167715,  0.47773084, -0.06143662],
        [ 0.57177788,  0.50622123, -0.03806645],
        ...,
        [        nan,         nan,         nan],
        [        nan,         nan,         nan],
        [        nan,         nan,         nan]],

       [[ 0.57099456,  0.55596769, -0.04227388],
        [ 0.56821048,  0.4816947 , -0.06146888],
        [ 0.56885695,  0.50883514, -0.03679752],
        ...,
        [        nan,         nan,         nan],
        [        nan,         nan,         nan],
        [        nan,         nan,         nan]],

       ...,

       [[ 0.5871107 ,  0.65902555, -0.0354271 ],
        [ 0

In [47]:
data.values

array([['face', 0, 1, nan, nan, nan],
       ['face', 1, 1, nan, nan, nan],
       ['face', 2, 1, nan, nan, nan],
       ...,
       ['right_hand', 18, 61, 0.6435529589653015, 1.008134126663208,
        -0.05265763774514198],
       ['right_hand', 19, 61, 0.6667584180831909, 1.0576053857803345,
        -0.037820637226104736],
       ['right_hand', 20, 61, 0.669672429561615, 1.1011139154434204,
        -0.024484368041157722]], dtype=object)

In [3]:
# def create_frame_landmark_df(results, frame, xyz):
#     xyz_skel = xyz[['type','landmark_index']].drop_duplicates().reset_index(drop=True).copy()

#     face = pd.DataFrame()
#     pose = pd.DataFrame()
#     left_hand = pd.DataFrame()
#     right_hand = pd.DataFrame()
#     if results.face_landmarks:
#         for i, point in enumerate(results.face_landmarks.landmark):
#             face.loc[i, ['x','y','z']] = [point.x, point.y, point.z]
#     if results.pose_landmarks:
#         for i , point in enumerate(results.pose_landmarks.landmark):
#             pose.loc[i, ['x', 'y', 'z']] = [point.x, point.y, point.z]
#     if results.left_hand_landmarks:
#         for i, point in enumerate(results.left_hand_landmarks.landmark):
#             left_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]
#     if results.right_hand_landmarks:
#         for i, point in enumerate(results.right_hand_landmarks.landmark):
#             right_hand.loc[i, ['x', 'y', 'z']] = [point.x, point.y, point.z]   
#     face = face.reset_index() \
#         .rename(columns={'index':'landmark_index'}) \
#             .assign(type='face')
#     pose = pose.reset_index() \
#         .rename(columns={'index':'landmark_index'}) \
#             .assign(type='pose')
#     left_hand = left_hand.reset_index() \
#         .rename(columns={'index':'landmark_index'}) \
#             .assign(type='left_hand')
#     right_hand = right_hand.reset_index() \
#         .rename(columns={'index':'landmark_index'}) \
#             .assign(type='right_hand')


#     landmarks = pd.concat([face,pose,right_hand,left_hand]).reset_index(drop=True)
#     landmarks = xyz_skel.merge(landmarks, on=['type','landmark_index'], how='left')
#     landmarks = landmarks.assign(frame=frame)
#     return landmarks



# def do_capture_loop(xyz):
#     all_landmarks = []
#     cap = cv2.VideoCapture(1)
#     with mp_holistic.Holistic(
#         min_detection_confidence=0.5,
#         min_tracking_confidence=0.5) as holistic:
#         frame = 0
#         while cap.isOpened():
#             frame+=1
#             success, image = cap.read()
#             if not success:
#                 print("Ignoring empty camera frame.")
#                 # If loading a video, use 'break' instead of 'continue'.
#                 continue

#         # To improve performance, optionally mark the image as not writeable to
#         # pass by reference.
#             image.flags.writeable = False
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             results = holistic.process(image)

#         ## create landmarks dataframe from results
#             landmarks = create_frame_landmark_df(results,frame,xyz)
#             all_landmarks.append(landmarks)
        
#         # Draw landmark annotation on the image.
#             image.flags.writeable = True
#             image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#             mp_drawing.draw_landmarks(
#                 image,
#                 results.face_landmarks,
#                 mp_holistic.FACEMESH_CONTOURS,
#                 landmark_drawing_spec=None,
#                 connection_drawing_spec=mp_drawing_styles
#                 .get_default_face_mesh_contours_style())
#             mp_drawing.draw_landmarks(
#                 image,
#                 results.pose_landmarks,
#                 mp_holistic.POSE_CONNECTIONS,
#                 landmark_drawing_spec=mp_drawing_styles
#                 .get_default_pose_landmarks_style())
#             #Flip the image horizontally for a selfie-view display.
            
#             # text = prediction_func(pq_file)
#             # print(text)
#             # draw_predictions(image,text)
#             # cv2.rectangle(frame, (x, y - text_height - 5), (x + text_width, y), (0, 0, 0), -1)
#             # cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

#             cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))

#             if cv2.waitKey(5) & 0xFF == 27:
#                 break
#     # cap.release()
#     return all_landmarks
# pq_file_sample = "train_landmark_files/16069/100015657.parquet"
# xyz = pd.read_parquet(pq_file_sample)
# landmarks = do_capture_loop(xyz)
# pd.concat(landmarks).reset_index(drop=True).to_parquet('output.parquet')